import packages

In [ ]:
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.distributions
import torchvision
import numpy as np

prepare/process data
 - normalize numerical col
 - create one hot encoding
 - create feature matric

In [2]:
data = pd.read_csv("cleaned_data.csv")
data.head()

,tconst,titleType,primaryTitle,isAdult,startYear,genres,directorNames,writerNames,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,0,1894,['Romance'],['Alexander Black'],['Alexander Black'],5.4,215
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,0,1897,"['Documentary', 'News', 'Sport']",['Enoch J. Rector'],[],5.2,538
2,tt0000502,movie,Bohemios,0,1905,[],['Ricardo de Baños'],"['Ricardo de Baños', ' Miguel de Palacios', ' ...",4.4,18
3,tt0000574,movie,The Story of the Kelly Gang,0,1906,"['Action', 'Adventure', 'Biography']",['Charles Tait'],['Charles Tait'],6.0,937
4,tt0000591,movie,The Prodigal Son,0,1907,['Drama'],['Michel Carré'],['Michel Carré'],5.7,28


create vae model design
 - encoder
 - decoder
 - define loss function

In [ ]:
class Encoder(nn.Module):

    def __init__( self, input_dim, hidden_dim, latent_dim ):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)      # Mean of latent distribution
        self.fc_log_var = nn.Linear(hidden_dim, latent_dim) # Log variance of latent distribution

    def forward(self, x):
        h = F.relu(self.fc1(x))
        mu = self.fc_mu(h)
        log_var = self.fc_log_var(h)
        return mu, log_var  # Return mean and log variance

class Decoder(nn.Module):

    def __init__( self, input_dim, hidden_dim, latent_dim ):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, input_dim)

    def forward(self, z):
        h = F.relu(self.fc1(z))  # Apply ReLU activation
        # might want to change .softmax to .sigmoid
        return torch.sigmoid(self.fc2(z))  # Return class probabilities

class PartialVAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(PartialVAE, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, latent_dim)
        self.decoder = Decoder(input_dim, hidden_dim, latent_dim)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)  # Standard deviation
        epsilon = torch.randn_like(std)  # Random noise
        return mu + epsilon * std  # Reparameterization trick

    def forward(self, x, mask):
        x_observed = x * mask #apply mask to only consider observed data
        mu, log_var = self.encoder(x_observed)  # Encode input to latent distribution
        z = self.reparameterize(mu, log_var)  # Sample latent variable
        x_reconstructed = self.decoder(z)  # Decode back to input space
        return x_reconstructed, mu, log_var  # Return reconstructed data, mean, and log variance

def loss_function(x_reconstructed, x, mu, log_var, mask):
    # Masked reconstruction loss (only consider observed values)
    reconstruction_loss = F.binary_cross_entropy(
        x_reconstructed * mask, x * mask, reduction="sum"
    )
    # KL divergence for latent space regularization
    kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return reconstruction_loss + kl_divergence


Training VAE
 - train model
  - forward pas
  - compute loss
  - backpropogation
  - run for # of epochs

In [ ]:
# Andrew

Generate Latent Representations
- evaluate user preferences
- compare with other movies

Content-Based Recomendations
 - Compute Similarities
 - generate reccomendations


Testing and Validations
 - Evaluate Reccomendations
 - Fine Tune Model